In [20]:
import pandas as pd

In [21]:
import math

In [22]:
import numpy as np

In [23]:
from scipy import stats

In [24]:
csv = pd.read_csv('./ocean157ksafe.csv', ',')

In [25]:
generos = {
    "1": "Mujer",
    "2": "Varón",
    "3": "No binarie",
    "4": "Género fluido",
    "5": "Ninguna de estas opciones me identifica",
    "6": "Prefiero no decirlo"
}

renames = {
    '1_extraversion': "Extroversión",
    '1_agreeableness': "Agradabilidad",
    '1_conscientiousness': "Responsabilidad",
    '1_neuroticism': "Neuroticismo",
    '1_openness': "Apertura a la experiencia",
    "2_genero": "Género",
}

traits = [
    "Extroversión",
    "Agradabilidad",
    "Responsabilidad",
    "Neuroticismo",
    "Apertura a la experiencia"
]

gen_keys = {
    "mujer": "mujeres",
    "varón": "varones",
    "no binarie": "noBinarie",
    "género fluido": "generoFluido"
}

In [26]:
csv = csv.rename(columns=renames)

In [27]:
for k in generos.keys():
    csv.loc[csv["Género"] == int(k), 'Género'] = generos.get(k)

In [50]:
def maxX(trait, area=0.9999):
    xs = []
    for genero in list(generos.values())[:4]:
        subset = csv.loc[csv["Género"] == genero][trait]
        valores = np.array(subset)
        f = stats.gaussian_kde(valores, 0.5)
        v = 0
        for i in range(0, 100):
            curr = f(i)[0]
            v = v + curr
            if abs(v - 1) < (1 - area):
                xs.append(i)
                break
    return max(xs)

In [112]:
"""
Para cada trait armar curvas para cada género
Queda algo como:
{
  "trait1": {
      "gen1": [
          [x0, y0],
          ...
          [xn, yn],
      ],
      ...
      "gen4": [
          [x0, y0],
          ...
          [xn, yn],
      ]
  },
  ...
  "trait5": { ... }
}
"""
trait_genero_curva = {}
for trait in traits: # para cada trait de OCEAN
    trait_genero_curva[f"{trait.lower()}"] = {}
    rangeMax = maxX(trait)
    xRange = range(0, maxX(trait) + 1)
    
    for genero in list(generos.values())[:4]: # para cada genero con suficientes respuestas
        subset = csv.loc[csv["Género"] == genero][trait]
        valores = np.array(subset)
        f = stats.gaussian_kde(valores, 0.5)
        trait_genero_curva[f"{trait.lower()}"][f"{gen_keys[genero.lower()]}"] = []
        for i in xRange:
            trait_genero_curva[
                f"{trait.lower()}"
            ][
                f"{gen_keys[genero.lower()]}"
            ].append([i, f(i)[0]])

In [115]:
"""
Parecido a lo anterior pero devuelve la misma distribución para todos los géneros.
Es para armar una curva de distribución unida que luego se va a separar
en los distintos grupos (géneros).
"""
joined = {}
# for trait in traits: # para cada trait de OCEAN
trait = "Apertura a la experiencia"
joined[f"{trait.lower()}"] = {}
rangeMax = maxX(trait)
xRange = range(0, maxX(trait) + 1)

for genero in list(generos.values())[:4]: # para cada genero con suficientes respuestas
    subset = csv[trait]
    valores = np.array(subset)
    f = stats.gaussian_kde(valores, 0.5)
    joined[f"{trait.lower()}"][f"{gen_keys[genero.lower()]}"] = []
    for i in xRange:
        joined[
            f"{trait.lower()}"
        ][
            f"{gen_keys[genero.lower()]}"
        ].append([i, f(i)[0]])